In [20]:
# Some steps required until these Python modules are properly installed...
#
import sys
import shutil
import glob
%pylab inline
import pandas as pd
import os
#
sys.path.append('../../../Modules')
sys.path.append('../../../../veneer-py')
# Get the Source scripting components (veneer) and GBR specific stuff
#
import veneer
import gbr

Populating the interactive namespace from numpy and matplotlib


In [21]:
### Declare variables of interest
#
Scenario_Directory = 'D:/a_sc_p2r_b_rc9/Model_runs/Burdekin Rebuild 2014'
Relevant_Scenarios = ['BU_RC9_BL_3_Hill_1']
Model_run_years = 31
Spatial_Output = 'D:/a_sc_p2r_b_rc9/Model_runs/Burdekin Rebuild 2014/RC9_Rainfall_Runoff.csv'

Region_intSCFU = "d:/a_sc_p2r_b_rc9/Configuration/BU_intSCFU.csv"

In [22]:
# Get an example scenario results set
#
gbr.init(Scenario_Directory)
results = gbr.Results('BU_RC9_BL_3_Hill_1')

In [23]:
# Get the climateTable as we are chasing hydro results
#
Hydro = results.get('climateTable')
Hydro = Hydro.reset_index()
# Remove zero values from climateTable, in order to remove redundant scenario place holder FU
#
Hydro = Hydro[Hydro>0].dropna()
Hydro [0:1]

,Catchment,FU,Element,Depth_m
0,SC #1608,Water,Rainfall,23.149846


In [24]:
# Pivots the raw hydro results as column
Hydro = Hydro.pivot_table(index = ['Catchment', 'FU'], columns = 'Element', values = 'Depth_m')
Hydro [0:1]

,Element,Actual ET,Baseflow,Rainfall,Runoff (QuickFlow)
Catchment,FU,,,,
SC #10,Conservation,24.394108,1.732494,35.730551,9.500099


In [25]:
# Average Annual / div by number of model run years
Hydro = Hydro/Model_run_years
# Conversion from (M) to (mm)
Hydro = Hydro*1000

In [26]:
# Calculates Total Runoff, Runoff as % of Rainfall, Baseflow as % of Total Runoff, QuickFlow as % of Total Runoff
Hydro['Total_Runoff'] = Hydro['Baseflow'] + Hydro['Runoff (QuickFlow)'] 
Hydro['Runoff_as_P_of_Rainfall'] = Hydro['Total_Runoff'] / Hydro['Rainfall'] *100
Hydro['Baseflow_as_P_of_T_Runoff'] = Hydro['Baseflow'] / Hydro['Total_Runoff'] *100
Hydro['Quickflow_as_P_of_T_Runoff'] = Hydro['Runoff (QuickFlow)'] / Hydro['Total_Runoff'] *100
Hydro [0:1]

,Element,Actual ET,Baseflow,Rainfall,Runoff (QuickFlow),Total_Runoff,Runoff_as_P_of_Rainfall,Baseflow_as_P_of_T_Runoff,Quickflow_as_P_of_T_Runoff
Catchment,FU,,,,,,,,
SC #10,Conservation,786.906709,55.886899,1152.598408,306.45479,362.341689,31.436942,15.423811,84.576189


In [27]:
# Read in intsFu table, remove superfluous columns and set Catchment FU cols as index 
#
Region_intSCFU = pd.read_csv(Region_intSCFU, error_bad_lines=False)
Region_intSCFU.rename(columns={'IntFUs':'FU','IntSCs':'Catchment'},inplace=True)
Region_intSCFU = Region_intSCFU.drop(['IntSCFU', 'FID'], axis=1)
Region_intSCFU.set_index(['Catchment','FU'],inplace=True)
Region_intSCFU [0:1]

,,IntSCFU_S
Catchment,FU,
SC #494,Grazing Forested,0


In [28]:
## Joins with regional INTSFU
#
Climate_Table = Region_intSCFU.join(Hydro)
Climate_Table [0:1]

,,IntSCFU_S,Actual ET,Baseflow,Rainfall,Runoff (QuickFlow),Total_Runoff,Runoff_as_P_of_Rainfall,Baseflow_as_P_of_T_Runoff,Quickflow_as_P_of_T_Runoff
Catchment,FU,,,,,,,,,
SC #494,Grazing Forested,0,577.167556,28.799791,741.750811,107.32309,136.12288,18.351565,21.1572,78.8428


In [29]:
# Exports To CSV
#
Climate_Table.to_csv(Spatial_Output)